## Setting up the data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
import warnings
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder


## Model Summary Class

In [ ]:
# ModelSummary Class
class ModelSummary:
    """
    This class extracts a summary of the model

    Methods
    ------
    get_se()
        Computes standard error
    get_ci(SE_est)
        Computes confidence intervals
    get_pvals()
        Computes p-values
    get_summary(name=None)
        Prints the summary of the model
    """
    def __init__(self, clf, X, y):
        """
        Parameters
        ---------
        clf: class
            The classifier object model
        X: pandas Dataframe
            Matrix of predictors
        y: numpy array
            Matrix of variable
        """
        self.clf = clf
        self.X = X
        self.y = y

    def get_se(self):
        self.clf.predict_proba(self.X)
        # From https://stats.stackexchange.com/questions/89484/how-to-compute-the-standard-errors-of-a-logistic-regressions-coefficient
        predProbs = self.clf.predict_proba(self.X)
        X_design = np.hstack([np.ones((self.X.shape[0], 1)), self.X])
        V = np.diagflat(np.product(predProbs, axis=1))
        covLogit = np.linalg.inv(np.dot(np.dot(X_design.T, V), X_design))
        return np.sqrt(np.diag(covLogit))

    def get_ci(self, SE_est):
        """
        Parameters
        ---------
        SE_est: numpy array
            Matrix of standard error estimations
        """
        p = 0.975
        df = len(self.X) - 2
        crit_t_value = stats.t.ppf(p, df)
        coefs = np.concatenate([self.clf.intercept_, self.clf.coef_[0]])
        upper = coefs + (crit_t_value * SE_est)
        lower = coefs - (crit_t_value * SE_est)
        cis = np.zeros((len(coefs), 2))
        cis[:, 0] = lower
        cis[:, 1] = upper
        return cis

    def get_pvals(self):
        self.clf.predict_proba(self.X)
        # From https://stackoverflow.com/questions/25122999/scikit-learn-how-to-check-coefficients-significance
        predProbs = self.clf.predict_proba(self.X)
        n = len(predProbs)
        m = len(self.clf.coef_[0]) + 1
        se = self.get_se()
        coefs = np.concatenate([self.clf.intercept_, self.clf.coef_[0]])
        t = coefs / se
        p = (1 - stats.norm.cdf(abs(t))) * 2
        return p

    def get_summary(self, names=None):
        ses = self.get_se()
        cis = self.get_ci(ses)
        lower = cis[:, 0]
        upper = cis[:, 1]
        pvals = self.get_pvals()
        coefs = np.concatenate([self.clf.intercept_, self.clf.coef_[0]])
        data = []
        for i in range(len(coefs)):
            currlist = [np.round(coefs[i], 3), np.round(ses[i], 3), np.round(pvals[i], 3), np.round(lower[i], 3), np.round(upper[i], 3)]
            data.append(currlist)
        cols = ['coefficient', 'std', 'p-value', '[0.025', '0.975]']
        sumdf = pd.DataFrame(columns=cols, data=data)
        if names is not None:
            new_names = ['intercept'] + names
            sumdf.index = new_names
        else:
            try:
                names = list(self.X.columns)
                new_names = ['intercept'] + names
                sumdf.index = new_names
            except:
                pass
        print(sumdf)
        acc = accuracy_score(self.y, self.clf.predict(self.X))
        confmat = confusion_matrix(self.y, self.clf.predict(self.X))
        print('-' * 60)
        print('Confusion Matrix (total:{}) \t Accuracy: \t  {}'.format(len(self.X), np.round(acc, 3)))
        print('  TP: {} | FN: {}'.format(confmat[1][1], confmat[1][0]))
        print('  FP: {} | TN: {}'.format(confmat[0][1], confmat[0][0]))


## Importing Data 

In [ ]:
# remove warnings
warnings.filterwarnings('ignore')

# reading the datase
data = pd.read_csv("heart_disease.csv")


## Feature Engineering

### Binarising dataset

In [ ]:
# Binarising Dataset
# Identify categorical features
categorical_features = ['cp', 'restecg', 'slope', 'thal']
# Perform one-hot encoding and drop the first category for features with more than 2 categories
data_encoded = pd.get_dummies(data, columns=categorical_features, drop_first=True)
data_encoded = data_encoded.drop('cp_typical angina', axis=1)
#Binarising to 0 and 1
data_encoded = data_encoded.replace({True: 1, False: 0})


### Correlation matrix

In [ ]:
# Compute the correlation matrix
correlation_matrix = data_encoded.corr()
# Print the correlation matrix
print(correlation_matrix)



### Removing unwanted features

In [ ]:
# Removing features unwanted after feature engineering
data_encoded=data_encoded.drop(['trestbps','chol','fbs','restecg_st-t  abnormality','slope_upsloping','thal_reversable defect'], axis=1)
data_encoded.head()


### Splitting dataset

In [ ]:
# splitting the independent and dependent variables
X = data_encoded.drop('num', axis = 1)
y = data_encoded['num']

# generating confusion matrix
mod = LogisticRegression()
mod.fit(X, y)
modsummary = ModelSummary(mod, X, y)
modsummary.get_summary()
total = len(data_encoded)

#positive and negative dataset
total = len(data_encoded)
pos = data_encoded['num'].sum() 
neg = total - pos 

# splitting by 70%, 30%
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=0)

# splitting by 70%, 15%, 15%
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=0)
print('The sizes for train, validation, test should be {}'.format((len(X_train), len(X_val), len(X_test))))


### Standandardising dataset

In [ ]:
# Standardize the datasets
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)


## Testing different kernel 

In [ ]:
# Default SVM model
model = SVC(probability=True, C=1, kernel='linear', gamma='scale')
model.fit(X_train, y_train)


# Model comparison with different kernels
## accuracy
accuracytrain = []
accuracyval = []
for k in ('linear', 'poly', 'rbf', 'sigmoid'):
    model = SVC(kernel=k, C = 1, gamma='scale')
    model.fit(X_train, y_train)
    print(k)
    y_predtrain = model.predict(X_train)
    accuracytrain.append(accuracy_score(y_train, y_predtrain))
    y_predval = model.predict(X_val)
    accuracyval.append(accuracy_score(y_val, y_predval))

kernelyaxis = ['linear', 'rbf', 'poly', 'sigmoid']
plt.plot(kernelyaxis, accuracytrain, 'g', label = 'Training')
plt.plot(kernelyaxis, accuracyval, 'b', label = 'Validation')
plt.xlabel('Accuracy')
plt.ylabel('Kernel')
plt.title('Accuracy Scores of Training Set (green) and Validation Set (blue) (C=1, gamma = scale)')


## precision
precisiontrain = []
precisionval = []
for k in ('linear', 'poly', 'rbf', 'sigmoid'):
    model = SVC(kernel=k, C = 1, gamma='scale')
    model.fit(X_train, y_train)
    print(k)
    y_predtrain = model.predict(X_train)
    precisiontrain.append(precision_score(y_train, y_predtrain))
    y_predval = model.predict(X_val)
    precisionval.append(precision_score(y_val, y_predval))

kernelyaxis = ['linear', 'rbf', 'poly', 'sigmoid']
plt.plot(kernelyaxis, precisiontrain, 'g', label = 'Training')
plt.plot(kernelyaxis, precisionval, 'b', label = 'Validation')
plt.xlabel('Precision')
plt.ylabel('Kernel')
plt.title('Precision Scores of Training Set (green) and Validation Set (blue) (C=1, gamma = scale)')

## recall
recalltrain = []
recallval = []
for k in ('linear', 'poly', 'rbf', 'sigmoid'):
    model = SVC(kernel=k, C = 1, gamma='scale')
    model.fit(X_train, y_train)
    print(k)
    y_predtrain = model.predict(X_train)
    recalltrain.append(recall_score(y_train, y_predtrain))
    y_predval = model.predict(X_val)
    recallval.append(recall_score(y_val, y_predval))

kernelyaxis = ['linear', 'rbf', 'poly', 'sigmoid']
plt.plot(kernelyaxis, recalltrain, 'g', label = 'Training')
plt.plot(kernelyaxis, recallval, 'b', label = 'Validation')
plt.xlabel('Recall')
plt.ylabel('Kernel')
plt.title('Recall Scores of Training Set (green) and Validation Set (blue) (C=1, gamma = scale)')


## Testing varying C-Value

In [ ]:
# Testing varying C
accuracy_train_c = []
accuracy_val_c = []
precision_train_c = []
precision_val_c = []
recall_train_c = []
recall_val_c = []
y_data_c = []
for c in np.arange(0.1, 10, 0.05):
    y_data_c.append(c)
    model = SVC(kernel='poly', C=c, gamma = 'scale')
    model.fit(X_train, y_train)
    y_predtrain = model.predict(X_train)
    accuracy_train_c.append(accuracy_score(y_train, y_predtrain))
    recall_train_c.append(recall_score(y_train, y_predtrain))
    precision_train_c.append(precision_score(y_train, y_predtrain))
    y_predval = model.predict(X_val)
    accuracy_val_c.append(accuracy_score(y_val, y_predval))
    recall_val_c.append(recall_score(y_val, y_predval))
    precision_val_c.append(precision_score(y_val, y_predval))

plt.plot(y_data_c, accuracy_train_c, 'g', label='Training Set')
plt.plot(y_data_c, accuracy_val_c, 'b', label='Validation Set')
plt.ylabel('Accuracy')
plt.xlabel('C')
plt.title('Accuracy Scores of Training Set and Validation Set, varying C')
plt.legend()
plt.show()

plt.plot(y_data_c, precision_train_c, 'g', label='Training Set')
plt.plot(y_data_c, precision_val_c, 'b', label='Validation Set')
plt.ylabel('Prediction')
plt.xlabel('C')
plt.title('Precision Scores of Training Set and Validation Set, varying C')
plt.legend()
plt.show()

plt.plot(y_data_c, recall_train_c, 'g', label='Training Set')
plt.plot(y_data_c, recall_val_c, 'b', label='Validation Set')
plt.ylabel('Recall')
plt.xlabel('C')
plt.title('Recall Scores of Training Set and Validation Set, varying C')
plt.legend()
plt.show()


## Testing varying gamma value

In [ ]:
# Training with varying gamma
accuracy_train_g = []
accuracy_val_g = []
precision_train_g = []
precision_val_g = []
recall_train_g = []
recall_val_g = []
y_data_g = []

for g in np.arange(0.001, 0.1, 0.001):
    y_data_g.append(g)
    model = SVC(kernel='poly', C=2, gamma = g)
    model.fit(X_train, y_train)
    y_predtrain = model.predict(X_train)
    accuracy_train_g.append(accuracy_score(y_train, y_predtrain))
    recall_train_g.append(recall_score(y_train, y_predtrain))
    precision_train_g.append(precision_score(y_train, y_predtrain))
    y_predval = model.predict(X_val)
    accuracy_val_g.append(accuracy_score(y_val, y_predval))
    recall_val_g.append(recall_score(y_val, y_predval))
    precision_val_g.append(precision_score(y_val, y_predval))

plt.plot(y_data_g, accuracy_train_g, 'g', label='Training Set')
plt.plot(y_data_g, accuracy_val_g, 'b', label='Validation Set')
plt.ylabel('Accuracy')
plt.xlabel('gamma')
plt.title('Accuracy Scores of Training Set and Validation Set, varying gamma')
plt.legend()
plt.show()

plt.plot(y_data_g, precision_train_g, 'g', label='Training Set')
plt.plot(y_data_g, precision_val_g, 'b', label='Validation Set')
plt.ylabel('Precision')
plt.xlabel('gamma')
plt.title('Precision Scores of Training Set and Validation Set, varying gamma')
plt.legend()
plt.show()

plt.plot(y_data_g, recall_train_g, 'g', label='Training Set')
plt.plot(y_data_g, recall_val_g, 'b', label='Validation Set')
plt.ylabel('Recall')
plt.xlabel('gamma')
plt.title('Recall Scores of Training Set and Validation Set, varying gamma')
plt.legend()
plt.show()


## Fianl model setup

In [ ]:
# Final model with best  value
model2 = SVC(probability=True, kernel='poly', C=2,  gamma = 0.05)
model2.fit(X_train, y_train)
y_predtr = model2.predict(X_train)
accuracytr = accuracy_score(y_train, y_predtr)
precisiontr = precision_score(y_train, y_predtr)
recalltr = recall_score(y_train, y_predtr)
print(f"training accuracy: {accuracytr}, training precision: {precisiontr}, training recall: {recalltr}")

y_pred = model2.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
print(f"validation accuracy: {accuracy}, validation precision: {precision}, validation recall: {recall}")

y_predt = model2.predict(X_test)
accuracyt = accuracy_score(y_test, y_predt)
precisiont = precision_score(y_test, y_predt)
recallt = recall_score(y_test, y_predt)
print(f" test accuracy: {accuracyt}, test precision: {precisiont}, test recall: {recallt}")

# Confusion matrix for test set
cm = confusion_matrix(y_test, y_predt)
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, cmap='Blues',annot=True)
ax.set_xlabel('Predicted')
ax.set_ylabel('Reality')
ax.set_title('Confusion Matrix(Test)')
ax.xaxis.set_ticklabels(['Negative', 'Positive'])
ax.yaxis.set_ticklabels(['Negative', 'Positive'])
plt.show()

recall = recall_score(y_test, y_predt)
print(f"recall: {recall}")


## PCA

In [ ]:
from sklearn.decomposition import PCA

#Rank the importance of the features with PCA
components = pca.components_
explained_variance = pca.explained_variance_ratio_
feature_importance = np.abs(components) * explained_variance.reshape(-1, 1)
feature_importance = feature_importance.sum(axis=0)
feature_ranking = np.argsort(feature_importance)[::-1]
# Print feature rankings
print("Feature ranking (most important first):")
for rank, index in enumerate(feature_ranking):
    print(f"Rank {rank + 1}: Feature {index} (importance = {feature_importance[index]:.4f})")



## Heat Map

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sb

heart = pd.read_csv('heart_disease_uci_2.csv')

# categorical variable into a binary 
# cp
heart['cp atypical angina'] = heart['cp'] == 'atypical angina'
heart['cp atypical angina'] = heart['cp atypical angina'].astype(int)

heart['cp non-anginal'] = heart['cp'] == 'non-anginal'
heart['cp non-anginal'] = heart['cp non-anginal'].astype(int)

heart['cp typical angina'] = heart['cp'] == 'typical angina'
heart['cp typical angina'] = heart['cp typical angina'].astype(int)

# restecg
heart['restecg normal'] = heart['restecg'] == 'normal'
heart['restecg normal'] = heart['restecg normal'].astype(int)

heart['restecg st-t abnormality'] = heart['restecg'] == 'st-t abnormality'
heart['restecg st-t abnormality'] = heart['restecg st-t abnormality'].astype(int)

# slope
heart['slope flat'] = heart['slope'] == 'flat'
heart['slope flat'] = heart['slope flat'].astype(int)

heart['slope upsloping'] = heart['slope'] == 'upsloping'
heart['slope upsloping'] = heart['slope upsloping'].astype(int)

# thal
heart['thal normal'] = heart['thal'] == 'normal'
heart['thal normal'] = heart['thal normal'].astype(int)
heart['thal reversable defect'] = heart['thal'] == 'reversable defect'
heart['thal reversable defect'] = heart['thal reversable defect'].astype(int)

del heart['cp']
del heart['restecg']
del heart['thal']
del heart['slope']

heart.head()

# plotting correlation heatmap 
dataplot = sb.heatmap(heart.corr())
# displaying heatmap 
plt.show()

plt.savefig('heatmap.png', dpi=300)
